# 概要
こんな人向け：機械学習初心者。モデル作成方法がわからな人。上位カーネルをコピペするだけになっちゃてる人。

pytorchで基本的なモデルを作成する方法を解説します。

※私の備忘録も兼ねているので間違っているかもしれません。

# 1. データ確認
コンペの内容は別のノートに書いているのでぜひご覧ください。(宣伝)

[https://www.kaggle.com/tomohiroh/ranzcr](http://)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/train.csv")

LABELS = [
    'ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
    'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
    'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
    'Swan Ganz Catheter Present'
]

df.head()

"StudyInstanceUID"から画像を引っ張ってきてモデルに学習させます。

In [ ]:
DEBUG = True
if DEBUG:
    df = df.sample(frac = 0.01).reset_index(drop = True)
    print(df.shape)

モデル作成だけが目的なのでデータ数を減らします。実際に提出する時はFalseにしましょう。

In [ ]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(df, test_size = 0.1)
print(train.shape, valid.shape)

train_test_splitで学習データ(勉強)と評価データ(答え合わせ)に分けます。

この分割方法は適当なので、高スコアを狙いたい方はGroupKFoldにしたりしましょう。

# 2. 画像読み込み

In [ ]:
train.head()

In [ ]:
path = train.iloc[0, 0]
path

画像のパスを１つ取り出しました。これから画像を読み込みます。

In [ ]:
path = "../input/ranzcr-clip-catheter-line-classification/train" + "/" + path + ".jpg"
path

学習用画像は"train"のフォルダにあります。

In [ ]:
import cv2
image = cv2.imread(path)
image.shape

cv2でパスから画像を読み取ります。

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image)
plt.show()

画像はレントゲン写真です。

In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

cv2で読み込むと色がBGR(青緑赤)の順に読み込まれるのでRGB(赤緑青)に変えます。

変換する意味があるかはよくわかっていません。すみません。

In [ ]:
path = train.iloc[1, 0]
path = "../input/ranzcr-clip-catheter-line-classification/train" + "/" + path + ".jpg"
image2 = cv2.imread(path)
image2.shape

２枚目の画像を読み込んでみます。何枚か確認するとわかりますが画像によってサイズが異なります。

サイズを統一しないと学習時にエラーが起きるので加工しましょう。

In [ ]:
from albumentations import Resize
dummy = Resize(width = 300, height = 300)(image = image)
dummy

albumentationsのResizeでサイズを変えます。

変換後のデータは辞書型になっているので"image"で取り出さないといけません。

In [ ]:
image = Resize(width = 300, height = 300)(image = image)["image"]
image.shape

300 x 300になっています。

In [ ]:
plt.imshow(image)
plt.show()

ちっちゃくなってスケールは維持されています。

In [ ]:
image2 = Resize(width = 300, height = 300)(image = image)["image"]
image2.shape

同じことを２枚目の画像に実行するとサイズが統一されているとわかります。

In [ ]:
from albumentations.pytorch import ToTensorV2
image = ToTensorV2()(image = image)["image"]
image.shape

pytorchに入れたいのでToTensorV2に入れます。さっきと同じく辞書型で返ってくるので"image"で取り出しましょう。

サイズを見ると色を示す３が先になっています。

pytorchの仕様で学習時は色が最初に来ていないとダメです。ココがめんどくさいですが変換しておきましょう。

# 3. Dataset
pytorchで学習する時はデータを取り出すシステムを作らないといけません。

データセットとデータローダーが必要です。先にデータセットを作ります。

In [ ]:
from torch.utils.data import Dataset

class TrainDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.studyuid = df["StudyInstanceUID"].values
        self.labels = df[LABELS].values
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        path = self.studyuid[idx]
        path = "../input/ranzcr-clip-catheter-line-classification/train" + "/" + path + ".jpg"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Resize(300, 300)(image = image)["image"]
        image = ToTensorV2()(image = image)["image"]
        labels = self.labels[idx]
        return image, labels

Datasetをインポートします。

データセットはクラスで作成します。

__init__：初期化条件。引数はtrainとかのデータフレームです。selfは必須なのでとりあえず書きましょう。

__len__：データサイズを定義するために必要。基本的に初期化時に渡したデータの行数です。

__getitem__：データを取り出すときに必要。indexが引数になります。

データを取り出すときはインデックスが引数になるので、例えば０が入ったときはstudyuidの１個目のパスが対象になります。

後はさっきまでと同じ処理を実行して画像データはimage、該当するラベル(正解)はlabelsとして出力されます。

In [ ]:
train_dataset = TrainDataset(train)
train_dataset[0]

データセットを作るときはpandasのデータフレームを渡します。これが__init__の引数となります。

実際に０を渡して最初のデータを見てみましょう。

最初に画像データが、次にラベルデータが出力されています。

In [ ]:
image, label = train_dataset[0]
plt.imshow(image.permute(1, 2, 0))
plt.show()
print(label)

このようにインデックスだけを使って画像とラベルと取り出すシステムができました。

# 4. DataLoader
作成したデータセットをデータローダーに入れます。

In [ ]:
from torch.utils.data import DataLoader

データローダーをインポートします。

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = 8, shuffle = True)

DataLoaderにデータセットを渡します。

ついでに以下の引数を指定します。

batch_size：一度に何枚取り出すか。多いほど学習が早いけどメモリを使う。小さいほどメモリを抑えるが学習に時間がかかり、１枚の特徴に大きな影響を受ける。

shuffle：ランダムな順で取り出す。

他にもdrop_lastなどがありますので本格的に学習させたい時は調べてください。

In [ ]:
for batch in train_loader:
    print(batch[0].shape)

for文で全データを取り出せます。

batch_sizeを８にしたので一度に８枚のデータが出力されています。

In [ ]:
valid_dataset = TrainDataset(valid)
valid_loader = DataLoader(valid_dataset, batch_size = 16, shuffle = False)

評価データでも同じことをします。

しかしshuffleは不要ですし学習ほど計算が重くないのでbatch_sizeも変えました。

# 5. モデル作成
モデルはEfficientNetを使います。

In [ ]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import timm
from pprint import pprint
pprint(timm.list_models(pretrained = True))

EfficientNetを利用する方法はいくつかあります。

今回はtimmという画像分類モデルのセットを使いました。Datasetにアップされているので"+Add data"からinputに入れておきましょう。

pip installで入れる方法もありますが、本コンペでは提出時にネットを接続できないので使えなかったりします。

In [ ]:
import torch.nn as nn

class Effnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.effnet = timm.create_model(model_name = "tf_efficientnet_b0", pretrained = False)
        n_features = self.effnet.classifier.in_features
        self.effnet.classifier = nn.Linear(n_features, len(LABELS))
    
    def forward(self, x):
        x = self.effnet(x)
        return x

torch.nnのModuleを使ってクラスを作成します。

superとinitは定型文なので気にせず書きましょう。

EfficientNetをtimm.create_modelで作成します。指定するモデル名はさっき出力したリストの中から選びます。

EfficientNetにはB0～B7まであり今回はB0です。

pretrained=Trueにすると学習済みモデルになりますが、ネットからパラメータをダウンロードする必要があるので、ネットOFFでは使えません。

最後の出力形式を変更したいので.classifierの部分をLinear(全結合層)に置き換えます。

この時の入力サイズが必要なのでn_featuresとして取得しておきましょう。出力サイズは予測したいLABELSの数です。

forwardは実際に学習(予測)するための関数です。入力をxとしてEfficientNetに通した結果を返します。

In [ ]:
model = Effnet()

変数modelとしてEfficientNetを作りました。

In [ ]:
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

計算にCPUを使うかGPUを使うか指定します。

右端の"setting"からGPUをONにできます。（現時点で週43時間無料）

ONにしているとtorch.cuda.is_availableがTrueになるのでDEVICEはcuda(GPUの種類)になります。FalseならCPUのまま。

In [ ]:
model = model.to(DEVICE)
print(DEVICE)

モデルに使用するCPUorGPUを設定します。忘れがち。

EfficientNetは計算が重いのでできればGPUにしましょう。

節約したい場合は、モデルの改善などデバッグ時はOFFにして本格的に学習させたい時にONにするといいです。

# 6. 学習
学習するステップは以下の通り。

・損失関数を決める

・最適化手法(optimizer)を決める

・train_loaderで学習させる

・valid_loaderで性能を確認する

In [ ]:
criterion = nn.BCEWithLogitsLoss()

損失関数です。出力結果を０～１の範囲(シグモイド関数)にスケールしてから予測との誤差みたいなものを計算します。

In [ ]:
optimizer = torch.optim.Adam(model.parameters())

最適化手法です。色々ありますがメジャーなAdamにしました。

さっき作ったモデル(model)のパラメータを渡しておきましょう。

In [ ]:
model.train()
for X, y in train_loader:
    optimizer.zero_grad()
    X = X.float().to(DEVICE)
    y = y.float().to(DEVICE)
    pred = model(X)
    loss = criterion(pred, y)
    loss.backward()
    optimizer.step()

最初に.trainで学習モードにします。何をしているのかはよくわかりません。

予測する前に一度.zero_gradでoptimizerをリセットします。

for文でtrain_loaderからデータを引っ張ってきます。Xとyとして取り出しました。

pytorchの学習時はfloat型にする必要があるので.floatで変換しましょう。

またモデルに入れるデータでもto(DEVICE)でCPUorGPUの設定が必要です。これも忘れがち。

modelにXを入れるとEfficientNetに通って予測ラベルとして出力されるので、損失関数に渡しましょう。

ここでは(予測, 正解)の順にします。逆に渡すと変な結果になるので注意。

すると誤差を計算してくれるので、次に.backward()でモデルに誤差を教えてあげます。

最後にoptimizerの.stepで最適化手法に従ってモデルが改善されます。

この処理をバッチの数だけ繰り返して１回の学習が終わります。

In [ ]:
model.eval()
valid_loss = 0
with torch.no_grad():
    for X, y in valid_loader:
        X = X.float().to(DEVICE)
        y = y.float().to(DEVICE)
        pred = model(X)
        loss = criterion(pred, y)
        valid_loss += loss.item()
valid_loss /= len(valid_loader)
print("Loss:", valid_loss)

評価データで性能を確認します。

最初に.evalで評価モードに変更します。これも何してるかわかりません。。。

評価時はモデルのパラメータを変更したくないのでtorch.no_gradでロックしておきます。

学習時と同じ要領でXyを取り出しfloat型にして予測させましょう。

次に損失関数を計算しますが今回はモデルに誤差を伝える(backward)ことは不要です。

全バッチでの誤差の平均をとりましょう。これが１回目の学習での性能です。

In [ ]:
for epoch in range(10):
    model.train()
    for X, y in train_loader:
        optimizer.zero_grad()
        X = X.float().to(DEVICE)
        y = y.float().to(DEVICE)
        pred = model(X)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
    model.eval()
    valid_loss = 0
    with torch.no_grad():
        for X, y in valid_loader:
            X = X.float().to(DEVICE)
            y = y.float().to(DEVICE)
            pred = model(X)
            loss = criterion(pred, y)
            valid_loss += loss.item()
    valid_loss /= len(valid_loader)
    print("Loss:", valid_loss)

同じことを複数回繰り返しました。

これでも十分なのですが毎回必ず誤差が小さくなるとは限りません。

つまり最後のモデルが最高の性能を出すとは限らないということです。

## ◆一連の流れまとめ◆

In [ ]:
model = Effnet().to(DEVICE)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

best_loss = np.inf
for epoch in range(10):
    model.train()
    for X, y in train_loader:
        optimizer.zero_grad()
        X = X.float().to(DEVICE)
        y = y.float().to(DEVICE)
        pred = model(X)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
    model.eval()
    valid_loss = 0
    with torch.no_grad():
        for X, y in valid_loader:
            X = X.float().to(DEVICE)
            y = y.float().to(DEVICE)
            pred = model(X)
            loss = criterion(pred, y)
            valid_loss += loss.item()
    valid_loss /= len(valid_loader)
    print(f"EPOCH:{epoch}, Loss:{valid_loss}")
    if valid_loss < best_loss:
        best_loss = valid_loss
        torch.save(model.state_dict(), "effnet.pth")
        print("saved...")

１度モデルを作り直したいのでこれまで実行したことをまとめました。

best_lossとして最小の誤差を定義します。スタートは無限大です。

もし評価データでの誤差がこれまでの最小誤差よりも小さかったら更新しましょう。ついでにモデルを保存します。

こうすると全ての学習が終わる頃には最も誤差の小さかったモデルが上書き保存されています。

これで学習は終わり。今回紹介したのは最低限モデルを構築するために必要なことだけです。

# 7. 予測

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.studyuid = df["StudyInstanceUID"].values
        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        path = self.studyuid[idx]
        path = "../input/ranzcr-clip-catheter-line-classification/test" + "/" + path + ".jpg"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Resize(300, 300)(image = image)["image"]
        image = ToTensorV2()(image = image)["image"]
        return image

test用のデータセットを作ります。学習用とほとんど同じ。

パスはtestのパスになっているので注意しましょう。

また正解ラベルを持たないので出力はimageのみです。

In [ ]:
test = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/sample_submission.csv")
test_dataset = TestDataset(test)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

データセットとデータローダーを定義しました。

主にvalid_loaderと同じことをしています。

In [ ]:
model = Effnet().to(DEVICE)
model.load_state_dict(torch.load("./effnet.pth"))

モデルを呼び出します。

In [ ]:
submit_preds = []

model.eval()
with torch.no_grad():
    for X in test_loader:
        X = X.float().to(DEVICE)
        submit_preds.append(model(X).sigmoid().to("cpu"))
    submit_preds = np.concatenate([p.numpy() for p in submit_preds], axis = 0)

test_loaderからデータをロードしてモデルに渡します。

出力される値を.sigmoidで０～１にスケールしましょう。

データをcpuに対応させないと後々エラーになるのでto("cpu")を付けておきます。

各バッチの予測結果をリスト(submit_preds)に入れておき、最後にnumpyの.concatenateで行方向(axis = 0)に結合します。

これで提出用の予測値ができました。

In [ ]:
submit = pd.DataFrame(submit_preds, columns = LABELS)
submit.head()

データフレームとして提出データを作成します。

さっきの予測結果を入れて列名はLABELSを利用しました。

In [ ]:
submit["StudyInstanceUID"] = test["StudyInstanceUID"]
submit = pd.concat([submit.iloc[:, -1], submit.iloc[:, :-1]], axis = 1)
submit.to_csv("submission.csv", index = False)

IDが１列目に必要なので足しておきます。.to_csvでCSVとして保存します。

index = Falseにしないと余計な列ができるので注意。

In [ ]:
print(test.shape, submit.shape)

一応サイズを確認。問題なさそうです。

右上の"Save Version"から保存してプレビュー画面下にある"output"から"submit"を押せば提出できます。

# 8. pytorchでやることまとめ
①データセット(init, len, getitem)とデータローダーを作る。

②モデルをクラスで定義する。forwardで学習する流れをくむ。

③損失関数(criterion)と最適化手法(optimizer)を決める。

④学習させる。(model.train, for ... loader, zero_grad, criterion(pred, 正解), backward, step)

⑤評価する。(model.eval)

⑥学習と評価を繰り返す(性能が良くなれば上書きする)

⑦提出用のデータセットとデータローダを作り、予測する